In [ ]:
# import torch
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from transformers import BertTokenizer, BertForSequenceClassification
# from torch.utils.data import Dataset, DataLoader
# import torch.nn as nn
# import torch.optim as optim
# from tqdm import tqdm
# import pickle

# # Load dataset
# df = pd.read_csv('final_data.csv')
# df = df.dropna(subset=['text'])
# df['text'] = df['text'].astype(str)

# # Encode labels
# label_encoder = LabelEncoder()
# df['label'] = label_encoder.fit_transform(df['intent'])
# pickle.dump(label_encoder, open('label_encoder.pkl', 'wb'))

# # Tokenize text
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# X_encodings = tokenizer(df['text'].tolist(), truncation=True, padding=True, max_length=32, return_tensors='pt')
# input_ids = X_encodings['input_ids']
# attention_mask = X_encodings['attention_mask']
# y = torch.tensor(df['label'].values)

# # Split data
# X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
#     input_ids, attention_mask, y, test_size=0.2, random_state=42
# )

# # Create Dataset and DataLoader
# class IntentDataset(Dataset):
#     def __init__(self, input_ids, attention_mask, labels):
#         self.input_ids = input_ids
#         self.attention_mask = attention_mask
#         self.labels = labels

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         return {
#             'input_ids': self.input_ids[idx],
#             'attention_mask': self.attention_mask[idx],
#             'labels': self.labels[idx]
#         }

# train_dataset = IntentDataset(X_train_ids, X_train_mask, y_train)
# test_dataset = IntentDataset(X_test_ids, X_test_mask, y_test)

# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=8)

# # Load model
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# # Train model
# epochs = 5
# for epoch in range(epochs):
#     model.train()
#     train_loss = 0
#     loop = tqdm(train_loader, leave=True)
#     for batch in loop:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         train_loss += loss.item()
#         loop.set_description(f'Epoch {epoch + 1}')
#         loop.set_postfix(loss=loss.item())

#     print(f"Epoch {epoch + 1}, Loss: {train_loss / len(train_loader)}")

# # Validate model
# model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         _, preds = torch.max(outputs.logits, dim=1)
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

# accuracy = correct / total
# print(f"Validation Accuracy: {accuracy * 100:.2f}%")


# start step by step

In [ ]:
# # import libraries
# import pandas as pd
# import torch
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader, Dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
# from transformers import get_scheduler
# from sklearn.metrics import accuracy_score, classification_report
# from tqdm import tqdm

# # Check GPU availability
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print("Device:", device)

# # Load dataset (replace with your actual dataset)
# df = pd.read_csv("../data/final_data.csv")  # Replace with your dataset file
# df = df[['text', 'intent']]  # Ensure columns are named 'text' and 'label'

# # Preprocess dataset (optional)
# def preprocess_text(text):
#     text = text.lower().strip()  # Lowercase and strip whitespace
#     return text

# df['text'] = df['text'].apply(preprocess_text)

# # Split into train and test
# train_texts, test_texts, train_labels, test_labels = train_test_split(
#     df['text'], df['intent'], test_size=0.2, random_state=42
# )

Device: cuda


In [ ]:
# # prepare dataset for pytorch
# class TextDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_len=128):
#         self.texts = texts
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts.iloc[idx]
#         label = self.labels.iloc[idx]

#         # Tokenize the text
#         encoding = self.tokenizer(
#             text,
#             max_length=self.max_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt"
#         )

#         return {
#             "input_ids": encoding["input_ids"].squeeze(0),
#             "attention_mask": encoding["attention_mask"].squeeze(0),
#             "label": torch.tensor(label, dtype=torch.long)
#         }

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

# # Create datasets
# train_dataset = TextDataset(train_texts, train_labels, tokenizer)
# test_dataset = TextDataset(test_texts, test_labels, tokenizer)

# # Create DataLoaders
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
# # Load pre-trained mBERT model
# model = AutoModelForSequenceClassification.from_pretrained(
#     'bert-base-multilingual-cased',
#     num_labels=len(df['label'].unique())  # Adjust to the number of classes
# ).to(device)


In [ ]:
# # define optimizer and scheduler
# # Define optimizer
# optimizer = AdamW(model.parameters(), lr=1e-5)

# # Define scheduler
# num_training_steps = len(train_loader) * 5  # 5 epochs
# lr_scheduler = get_scheduler(
#     "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
# )


In [ ]:
# # train model in Training loop
# epochs = 5
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}")

#     for batch in progress_bar:
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["label"].to(device)

#         # Forward pass
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss

#         # Backward pass
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         lr_scheduler.step()

#         total_loss += loss.item()
#         progress_bar.set_postfix({"loss": total_loss / len(train_loader)})

#     print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


In [ ]:
# # evaluate model
# # Evaluation loop
# def evaluate_model(model, data_loader):
#     model.eval()
#     all_labels = []
#     all_preds = []

#     with torch.no_grad():
#         for batch in data_loader:
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             labels = batch["label"].to(device)

#             outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#             preds = torch.argmax(outputs.logits, dim=-1)

#             all_labels.extend(labels.cpu().numpy())
#             all_preds.extend(preds.cpu().numpy())

#     accuracy = accuracy_score(all_labels, all_preds)
#     print("Accuracy:", accuracy)
#     print("Classification Report:")
#     print(classification_report(all_labels, all_preds))

# # Evaluate the model
# evaluate_model(model, test_loader)


In [ ]:
# # save model
# model.save_pretrained("./mb_model")
# tokenizer.save_pretrained("./mb_model")
# print("Model saved successfully!")

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import pickle

# Ensure PyTorch runs on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device used: {device}")

# Load dataset
df = pd.read_csv('../data/final_data.csv')  # Ensure it contains 'text' and 'intent' columns

# Drop NaN values and convert text to strings
df = df.dropna(subset=['text'])
df['text'] = df['text'].astype(str)

# Encode intent
intent_encoder = LabelEncoder()
df['intent'] = intent_encoder.fit_transform(df['intent'])
pickle.dump(intent_encoder, open('intentencoder.pkl', 'wb'))  # Save encoder

# Ensure intent are within the valid range
num_intent = len(df['intent'].unique())
print(f"Number of unique intents: {num_intent}")

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text
X = df['text'].tolist()
y = df['intent'].values

X_encodings = tokenizer(X, truncation=True, padding=True, max_length=32, return_tensors='pt')

# Extract input_ids and attention_mask
input_ids = X_encodings['input_ids']
attention_mask = X_encodings['attention_mask']

# Split data
X_train_ids, X_test_ids, y_train, y_test = train_test_split(input_ids, y, test_size=0.2, random_state=42)
X_train_mask, X_test_mask = train_test_split(attention_mask, test_size=0.2, random_state=42)

# Custom Dataset class for PyTorch
class IntentDataset(Dataset):
    def __init__(self, input_ids, attention_mask, intent):
        self.input_ids = torch.tensor(input_ids, dtype=torch.long)
        self.attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        self.intent = torch.tensor(intent, dtype=torch.long)

    def __len__(self):
        return len(self.intent)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'intent': self.intent[idx]
        }

# Create PyTorch DataLoader
train_dataset = IntentDataset(X_train_ids, X_train_mask, y_train)
test_dataset = IntentDataset(X_test_ids, X_test_mask, y_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intent)
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define training loop
def train(model, train_loader, optimizer, epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        intent = batch['intent'].to(device)

        # Corrected model call
        outputs = model(input_ids, attention_mask=attention_mask, labels=intent)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(logits, dim=1)
        total += intent.size(0)
        correct += (predicted == intent).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch {epoch}: Loss = {avg_loss:.4f}, Accuracy = {accuracy * 100:.2f}%")

# Define evaluation loop
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            intent = batch['intent'].to(device)

            # Corrected model call
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            _, predicted = torch.max(logits, dim=1)
            total += intent.size(0)
            correct += (predicted == intent).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy = {accuracy * 100:.2f}%")

# Train the model for 5 epochs
for epoch in range(5):
    print(f"Epoch {epoch + 1}/5")
    train(model, train_loader, optimizer, epoch)
    evaluate(model, test_loader)

# Save the model and tokenizer
model.save_pretrained('../model/intent_classifier_bert')
tokenizer.save_pretrained('../model/bert_tokenizer')
print("Model and tokenizer saved successfully!")


c:\Users\M\miniconda3\envs\cudaenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device used: cuda
Number of unique intents: 4


C:\Users\M\AppData\Local\Temp\ipykernel_25304\2073034326.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids = torch.tensor(input_ids, dtype=torch.long)
C:\Users\M\AppData\Local\Temp\ipykernel_25304\2073034326.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.attention_mask = torch.tensor(attention_mask, dtype=torch.long)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\M\miniconda3\envs\cudaenv\lib\site-packages\tr

Epoch 1/5
Epoch 0: Loss = 1.2283, Accuracy = 44.44%
Test Accuracy = 61.76%
Epoch 2/5
Epoch 1: Loss = 0.8706, Accuracy = 82.96%
Test Accuracy = 91.18%
Epoch 3/5
Epoch 2: Loss = 0.5311, Accuracy = 97.04%
Test Accuracy = 94.12%
Epoch 4/5
Epoch 3: Loss = 0.2837, Accuracy = 100.00%
Test Accuracy = 94.12%
Epoch 5/5
Epoch 4: Loss = 0.1618, Accuracy = 99.26%
Test Accuracy = 97.06%
Model and tokenizer saved successfully!


In [6]:
print("Unique labels:", np.unique(y))

Unique labels: [0 1 2 3]
